In [1]:
# Import important library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Q1 Read the input file and check the data dimension

In [2]:
#Read input file and understand the data
# "default" is my dependent variable
data = pd.read_csv('german_credit.csv')
df_credit=data.dropna()
print("Input Data and Shape")
print(df_credit.shape)
print(df_credit.head())

#df_credit.dtypes

Input Data and Shape
(1000, 21)
   default account_check_status  duration_in_month  \
0        0               < 0 DM                  6   
1        1    0 <= ... < 200 DM                 48   
2        0  no checking account                 12   
3        0               < 0 DM                 42   
4        1               < 0 DM                 24   

                                      credit_history  \
0  critical account/ other credits existing (not ...   
1           existing credits paid back duly till now   
2  critical account/ other credits existing (not ...   
3           existing credits paid back duly till now   
4                    delay in paying off in the past   

                        purpose  credit_amount                      savings  \
0           domestic appliances           1169  unknown/ no savings account   
1           domestic appliances           5951                 ... < 100 DM   
2  (vacation - does not exist?)           2096                 ... < 

In [3]:
df_credit.head()
print(df_credit.columns)

Index(['default', 'account_check_status', 'duration_in_month',
       'credit_history', 'purpose', 'credit_amount', 'savings',
       'present_emp_since', 'installment_as_income_perc',
       'personal_status_sex', 'other_debtors', 'present_res_since', 'property',
       'age', 'other_installment_plans', 'housing', 'credits_this_bank', 'job',
       'people_under_maintenance', 'telephone', 'foreign_worker'],
      dtype='object')


In [4]:
# Lets build a Ensemble model but need to modify the dataset first


### Q2.Prepare the model data by converting non-numeric to dummy ( 1 Marks)
##### Hint: Use get_dummies

In [68]:
# Print Shape of model data
df_dummies=pd.get_dummies(df_credit)
print(df_dummies.head())

#df = df_credit.loc[:, df_credit.columns.intersection(['default','duration_in_month','credit_amount','age'])]
#print(df)


   default  duration_in_month  credit_amount  installment_as_income_perc  \
0        0                  6           1169                           4   
1        1                 48           5951                           2   
2        0                 12           2096                           2   
3        0                 42           7882                           2   
4        1                 24           4870                           3   

   present_res_since  age  credits_this_bank  people_under_maintenance  \
0                  4   67                  2                         1   
1                  2   22                  1                         1   
2                  3   49                  1                         2   
3                  4   45                  1                         2   
4                  4   53                  2                         2   

   account_check_status_0 <= ... < 200 DM  account_check_status_< 0 DM  ...  \
0                  

### Check for highly correlated variables but don't required any treatment for this use case

In [6]:
df_credit.corr()

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance
default,1.000000,0.214927,0.154739,0.072404,0.002967,-0.091127,-0.045732,-0.003015
duration_in_month,0.214927,1.000000,0.624984,0.074749,0.034067,-0.036136,-0.011284,-0.023834
credit_amount,0.154739,0.624984,1.000000,-0.271316,0.028926,0.032716,0.020795,0.017142
installment_as_income_perc,0.072404,0.074749,-0.271316,1.000000,0.049302,0.058266,0.021669,-0.071207
present_res_since,0.002967,0.034067,0.028926,0.049302,1.000000,0.266419,0.089625,0.042643
age,-0.091127,-0.036136,0.032716,0.058266,0.266419,1.000000,0.149254,0.118201
credits_this_bank,-0.045732,-0.011284,0.020795,0.021669,0.089625,0.149254,1.000000,0.109667
people_under_maintenance,-0.003015,-0.023834,0.017142,-0.071207,0.042643,0.118201,0.109667,1.000000


### Drop the original variables which are converted to dummy

In [7]:
# The below python code intersection will retain only the collumns mentioned inside brackets [] and drop the rest.
df_credit_new=df_credit.loc[:, df_credit.columns.intersection(['default','duration_in_month','credit_amount','age'])]
print(df_credit_new)

     default  duration_in_month  credit_amount  age
0          0                  6           1169   67
1          1                 48           5951   22
2          0                 12           2096   49
3          0                 42           7882   45
4          1                 24           4870   53
5          0                 36           9055   35
6          0                 24           2835   53
7          0                 36           6948   35
8          0                 12           3059   61
9          1                 30           5234   28
10         1                 12           1295   25
11         1                 48           4308   24
12         0                 12           1567   22
13         1                 24           1199   60
14         0                 15           1403   28
15         1                 24           1282   32
16         0                 24           2424   53
17         0                 30           8072   25
18         1

### Q3 Split Train/Test data 70:30 ratio( 1 Marks)
##### Hint:from sklearn.model_selection import train_test_split

In [50]:
from sklearn.model_selection import train_test_split
array = df_credit_new.values
X = array[:,1:3] # select all rows and columns except the 0th column which is the defaulter's column.
Y = array[:,0]   # select all rows and the 0 th column which is the classification "Yes", "No" for defaulters
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [31]:
# This is just for testing and understanding how good a single Decision tree is in comparison 500 decision tree in a Random Forest
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(criterion = 'entropy' )
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [54]:
# This code is only for the purpose of understanding.
from sklearn import metrics
print("Score on the Test Set for single Decision Tree is \n",dt_model.score(X_test , y_test))
print("Score on the Train dataset for Single Decision Tree  is \n",dt_model.score(X_train,y_train))
# Printing the confusion matrix for a single Decision Tree
# this produces a 2x2 numpy array (matrix)
y_predict = dt_model.predict(X_test)
print(metrics.confusion_matrix(y_test, y_predict))

Score on the Test Set for single Decision Tree is 
 0.6166666666666667
Score on the Train dataset for Single Decision Tree  is 
 1.0
[[158  59]
 [ 56  27]]


### Q4 Build Random Forest Model( 1 Marks)
#### Hint:from sklearn.ensemble import RandomForestClassifier using n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999

In [55]:
from sklearn.ensemble import RandomForestClassifier 
rf_model=RandomForestClassifier(n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999)
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
            oob_score=False, random_state=9999, verbose=0,
            warm_start=False)

### Q5 Calculate Confusion Matrix and Accuracy score (1 Marks)
##### Hint: Use confusion_matrix and accuracy_score

In [56]:
from sklearn import metrics
print("Accuracy score on the Test dataset is \n",rf_model.score(X_test , y_test))
print("Accuracy Score on Train dataset is \n",rf_model.score(X_train,y_train))
# first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
y_predict = rf_model.predict(X_test)
print(metrics.confusion_matrix(y_test, y_predict))

Accuracy score on the Test dataset is 
 0.6466666666666666
Accuracy Score on Train dataset is 
 1.0
[[164  53]
 [ 53  30]]


### Q6 Show the list of the features importance( 1 Marks)

In [57]:
print (pd.DataFrame(rf_model.feature_importances_))

          0
0  0.150009
1  0.849991


### Q7 K-fold cross-validation( 2 Marks)
##### k-fold cross validation( without stratification)
##### Usually k is set as 10-20 in practical settings, depends on data set size

In [58]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [59]:
# Use below values
num_folds = 10
seed = 77

In [14]:
#Validate the Random Forest model build above using k fold

In [60]:
kf = KFold(n_splits=10) # Define the split - into 10 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 
KFold(n_splits=num_folds, random_state=seed, shuffle=False)

KFold(n_splits=10, random_state=None, shuffle=False)


KFold(n_splits=10, random_state=77, shuffle=False)

In [15]:
#Calculate Mean score

In [61]:
scores=cross_val_score(rf_model,X,Y,cv=10)
scores.mean()

0.63

In [17]:
# Calculate score standard deviation using std()

In [62]:
scores.std()

0.050793700396801186

# Q8 Print the confusion matrix( 1 Marks)

In [63]:
print(metrics.confusion_matrix(y_test, y_predict))

[[164  53]
 [ 53  30]]


# Q9.Classification accuracy: 
percentage of correct predictions and Calculate sensitivity (or True Positive Rate or Recall) and Precision.
(2 Marks)

In [21]:
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.70      0.73      0.71       207
           1       0.33      0.29      0.31        93

   micro avg       0.59      0.59      0.59       300
   macro avg       0.51      0.51      0.51       300
weighted avg       0.58      0.59      0.59       300



[link text](https://)### Bootstrapping ( Bonus and Optional)
##### Given a dataset of size n, a bootstrap sample is created by sampling n instances uniformly from the data (with/without replacement)
##### Create a model with each bootstrap sample and validate it with the test set
##### Final result is calculated by averaging the accuracy of models

In [64]:
# Number of iterations for bootstrapping
# Using the same german credit dataset of 1000 records for this.
bootstrap_iteration = 10
accuracy = []

In [65]:
# Using the same Xtrain and y_train values
#print(X_train,y_train)

In [66]:
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
# Resampling X_train and y_train data and generating the new data X_ and y_ for prediction
for i in range(bootstrap_iteration):
    X_, y_ = resample(X_train, y_train)
    rfm.fit(X_, y_)
    y_pred = rfm.predict(X_test)    
    acc = accuracy_score(y_pred, y_test)
    accuracy.append(acc)

In [67]:
accuracy = np.array(accuracy)
print('Accuracy Score')
print('Avearge: ', accuracy.mean())
print('Standard deviation: ', accuracy.std())

Accuracy Score
Avearge:  0.6379999999999999
Standard deviation:  0.024413111231467426
